### Complet à verifier

In [6]:
# ------------------------------------------------------------------------ #  
# ------------------------------------------------------------------------ #
# ------------------------------------------------------------------------ #
# ------------------------------------------------------------------------ #
#####
### CONTROLES
#####
    
def v(t):
    out = 2 + (t>5)*(t<10)
    return out

def u(t):
    out = -0.22*t+(0.44*t)*(t>5)-(0.5*(t-5))*(t>9)+(0.5*(t-13))*(t>13)+(0.2*(t-8))*(t>15) 
    return out

### Complet à verifier

In [ ]:
#####
### CALCUL DE RHO_2
#####


def AK(z1,z2,z3,B):
# Determination de rho2 en fonction de la distance AB, de l'angle xsi et de
# rho1
#
# formule obtenue ? patir de la formule d'Al-Kashi, valable uniquement
# lorsque x \in [xA;xB]
#
    radicande = (B[0]**2-(z2*math.sin(z3-z1))**2)*(B[0]**2-(z2*math.sin(z3-z1))**2>0)

    
    out = z2*math.cos(z3-z1)+math.sqrt(radicande)
    return out


###  Terminer la conversion, voir comment gérer les logicals et verifier les y(S)

In [ ]:
#####
### Fonction SymReshape
#####


# Attention verfication conversion python


def SymReshape(u):
# SymReshape     Reshape dedicated to a symetric matrix
#   if u is a symetric matrix, y will be a vector with n(n+1)/2 components
#   obtained from column of u without repeating symetric values.
#   if u is a vector with n(n+1)/2 components, y will be the corresponding
#   symetric matrix.

[m,n]=u.shape
if (m==n):
   S=logical(np.reshape(np.tril(np.ones((m,n))),m*n,1));
   y=u(S)
else:
   n=(math.sqrt(1+8*m)-1)/2
   y=np.zeros(n,n)
   S=logical(np.reshape(np.tril(np.ones((n,n))),n**2,1));
   y(S)=u
   y=np.transpose(y)
   y(S)=u

return y

### Complet à verifier

In [ ]:
#
#
#####
### MODELE SOUS FORME NORMALE
#####
#
#


def A_mat(t):
    out = np.array([[0,           0,          0], 
                    [0,           0,          0],
                    [0,           0,          0]])
    return out

def b_mat(t,z):
    global B
   
    AK = AK(z[0],z[1],z[2],B)   
    
    out=np.array([v(t)*math.sin(z[0]/z[1]-u(t)],
                 [-v(t)*math.cos(z[0])],
                [v(t)*math.sin(z[2])/AK-u(t)])

    return out


### Complet à verifier

In [ ]:
def Db_mat(t,z):

    global B

    xb=B[1]
    xi=z[2]-z[0]

    den = AK(z[0],z[1],z[2],B)

    AA=-v(t)*math.sin(z[2])*(z[1]*math.sin(xi)+z[1]**2*math.sin(xi)*math.cos(xi)/math.sqrt(abs(xb**2-(z[1]*math.sin(xi))**2)))/den**2

    BB=-v(t)*math.sin(z[2])*(math.cos(xi)-z[1]*math.sin(xi)**2/math.sqrt(abs(xb**2-(z[1]*math.sin(xi))**2)))/den**2
    
    C1=v(t)*math.cos(z[2])/den 
    C2=v(t)*math.sin(z[2])*(z[1]*math.sin(xi)+z[1]**2*math.cos(xi)*math.sin(xi)/math.sqrt(abs(xb**2-(z[1]*math.sin(xi))**2)))/den**2
    
    CC=C1+C2

    
    out= np.array([[v(t)*math.cos(z[0])/z[1],   -v(t)*math.sin(z[0])/(z[1]**2),    0],
                  [v(t)*math.sin(z[0]),                   0,                       0],
                  [ AA,                                   BB,                    CC ]])

    return out


### Complet à verifier, lancer avant SymReshape, A_mat, b_mat et Db_mat

In [ ]:
#####
### KALMAN ETAPE DE PREDICTION
#####


def Kalman_Pred_Step(t,X,theta, Qtheta):
# Prediction step of the continuous-discrete asynchronous K filter
#   
### Retrieve state / estimation / Riccati equation
    n = 3
    nRic = n*(n+1)/2
# --- # 
    Xhat = X[0:n]
    S = SymReshape(X[n:n+nRic])
# ----- #    
    # No Output related Data
# ----- #
    dXhat = A_mat(t)* Xhat + b_mat(t,Xhat)    #-P*C'*IR_t*C*(Xhat-mes(:,where));                    #etat estime 
# --- #   
    dS =-np.transpose(A_mat(t)+Db_mat(t,Xhat))*S-S*np.transpose(A_mat(t)+Db_mat(t,Xhat))-S*Qtheta*S #matrice de Riccati
# --- #
    out = np.concatenate((dXhat,SymReshape(dS)), axis=0)
    return out

In [7]:
#def Kalman_asynchrone_NB():
import numpy as np
import string
#from math import *
import math
from scipy.integrate import odeint
import matplotlib.pyplot as pypl


#global B

###      ###
# Couleurs #
###      ###
C_trajectory = np.array([0, 0, 0.5]) # Bleu Navy
C_rotating_Part = np.array([0, 100/255, 0]) # Vert Sombre
C_Boat2Amer = np.array([176/255, 196/255,  222/255]) # bleu acier clair
C_amer1 = np.array([255/255,20/255,147/255]) # Rose intense
C_amer2 = np.array([210/255,105/255,30/255]) # Chocolat

###        ###
# Parametres #
###        ###
#
#si Display=0: la simulation de la trajectoire du bateau n'apparait pas. 
#Si Display=1: Affichage de la trajectoire
Display=False



###
#  Options de ode
#   options = odeset('RelTol',1e-10,'AbsTol',1e-10,'MaxStep',1e-2)

A=np.array([[0],[0]])     # balise A1
B=np.array([[14],[0]])   # balise A2
tend = 18
t=np.arange(0,tend+0.005,0.005)


###                              ###
# X=(x,y,theta) et \dot{X}=AX+B(u) #
###         
    
    ###
    
def f(x0,t,u,v):
    u0=u(t)
    v0=v(t)
    x1, x2, x3=x0
    dfdt=[v0*np.cos(x3),v0*np.sin(x3), u0]
    return dfdt   #f(y,t)=y 

def TheBoat(x,y,theta):
    Xboat =np.array([ -0.3750,    0.3750,    0.7000,    0.3750,   -0.3750,  -0.3750])
    Yboat = np.array([0.1,  0.1,  0,  -0.1,  -0.1,  0.1])
 
    Rotation = np.array([[  np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta) ]]) 
    out = Rotation.dot(np.array([Xboat,Yboat])) +  np.array([[x],[y]])*(np.ones(6))
                                              
    return out

#t = np.linspace(0,18,1000)
x0=np.array([1,6,1])

XOUT = odeint(f,x0,t,args=(u,v))


    #XOUT(:,1) correspond a x(t), XOUT(:,2) correspond a y(t),XOUT(:,3) correspond a theta(t)

#XOUT[:,2]= math.fmod(XOUT[:,2],(2*math.pi));
for i in range(0,len(t)):
    XOUT[i,2]=(XOUT[i,2]%(2*math.pi))
    #print('i=',i,', y[i]=',y1[i],', x[i]=',x[i])
    

ya=np.zeros(len(t))
yb=np.zeros(len(t))
    
for i in range(0,len(t)):
        #mesure de l'angle entre l'oriantation du bateau et la balise A
    ya[i]=(math.atan2((A[1]-XOUT[i,1]),(A[0]-XOUT[i,0]))-XOUT[i,2])%(2*math.pi);

        #mesure de l'angle entre l'oriantation du bateau et la balise B
    yb[i]= (math.atan2((B[1]-XOUT[i,1]),(B[0]-XOUT[i,0]))-XOUT[i,2])%(2*math.pi);
    
print(ya)
print(yb)
    
    
##### ------------------------------------------------------  #####
# Computation Asynchronous mesurements
#
#
#
#  
w=10 #vitesse de rotation du capteur angulaire 
Angle_temp=np.zeros(len(t))
for i in range(0,len(t)):
    Angle_temp[i]=(w*t[i])%(2*math.pi)      #  Angle_temp(:,1)=mod(w*t(1,:),2*pi);

print("ya",ya,"\n yb",yb)
print("angle temp:",Angle_temp, len(Angle_temp))
S1=[]
S2=[]
ii=[]
for i in range (0,len(t)-1):
    if (math.fabs(ya[i]-Angle_temp[i])<1e-1):
        S1.append( t[i])
        ii.append(i)
        #print('OK')
    if (math.fabs(yb[i]-Angle_temp[i])<1e-1):
        S2.append( t[i])
        ii.append(i)
        #print('OK')


print('S1', len(S1))
print('S2', len(S2))
print('max(ya)',max(ya),'\n','min(ya)',min(ya),'\n','max(yb)',max(yb),'\n','min(yb)',min(yb),'\n','moyenne ya', np.mean(ya),'\n','moyenne yb', np.mean(yb))

####   Next - Remove Multi-instances #####
        
#for k in range(0,len(S1)-1):
#    if (S1[k+1]-S1[k]<1e-1):
#        S1[k+1]=(S1[k]+S1[k+1])/2
#        S1[k]=0
#print('S1=',S1)




#fonction qui fait la moyenne des elements d une liste dont la difference 
#est inferieur a epsilon et les revoie sous forme de liste
def rm_multi_instance(S1,epsilon):     
    S2 = [] 
    S3 = []
    i=0
    
    while i<len(S1):
        S2=[] 
        S2.append(S1[i])

        if i+1<len(S1):

            while abs(S1[i+1]-S1[i]) <= epsilon:

                S2.append(S1[i+1])
                i+=1

                if i>=(len(S1)-1):
                    break

        S3.append(np.mean(S2))

        i+=1
        
    return S3    

print('\n\nS1 avant remove',S1, '\n\nlen S1', len(S1))

S1=(rm_multi_instance(S1,1e-1)).copy()
print('\n\nS1 apres remove',S1, '\n\nlen S1', len(S1), '\n\n')


print('\n\nS2 avant remove',S2, '\n\nlen S2', len(S2))

S2=(rm_multi_instance(S2,1e-1)).copy()
print('\n\nS2 apres remove',S2, '\n\nlen S2', len(S2), '\n\n')





#temp1=np.where(S1 != 0)#[0]                         ## temp1=find(S1);
temp1=[]
temp2=[]
for i in range(0,len(S1)):
    if (S1[i]!=0):
        temp1.append(i)
        temp2.append(S1[i])


#temp2=np.zeros(len(S1))
print('taille de temp1:',len(temp1),'\n', 'temp1:', temp1, '\n', 'taille de temp2:', len(temp2),'\n', 'temp2:',temp2)


#for i in range(0,len(temp1)):
#    temp2[i]=S1[temp1[i]];
#S1=np.zeros(len(temp2));
#S1=temp2;
 
for k in range(0,len(S2)-2):
    if (S2[k+1]-S2[k]<1e-1):
        S2[k+1]=(S2[k]+S2[k+1])/2
        S2[k]=0
        #print('test2')


#for i in range(0,len(temp1)):
    #temp2[i]=S1[temp1[i]];
#S1=np.zeros(len(temp2));
#S1=temp2;

#temp2=np.where(S2 == 0)[0]  
#for i in range(0,len(temp1)-1):
#    temp2[i]=S2[temp1[i]]
    #print('test3')

S2=np.zeros(len(temp2))
S2=temp2.copy()
print('S1', len(S1))
print('S2', len(S2))


#         END % Computation Asynchronous mesurements              #
##### ------------------------------------------------------  #####
 
 
 
##### ------------------------------------------------------ #####
# si <Display = 1>
# Pour tracer la trajectoire du bateau "en temps reel+ le capteur qui
# tourne+la droite qui relie le bateau -> la balise.
 
if Display: 
    FigHandle=pypl.figure(1)       #,figsize=(8,6), dpi=80)
    for i in range(0,len(t)):
        pypl.clf
        #hold on
        #pypl.gca().set_aspect('equal', adjustable='box')        #axis('square')
        pypl.xlim(-2, 15)
        pypl.ylim(-1, 13)                                         #axis([-2,12,-5,12])
        ###      pypl.plot([A[0], B[0]],[A[1], B[1]],'--','Color','k')
        pypl.plot(A[0],A[1],marker='o')#,color='C_amer1')
        pypl.plot(B[0],B[1],marker='o')#,color='C_amer2')
        #pypl.plot(A[0,0], A[1,0],'o','Color',C_amer1)
        #pypl.plot(B[0,0], B[1,0],'o','Color',C_amer2)
 
        pypl.plot(XOUT[:,0], XOUT[:,1],linewidth='0.2')#,color='C_trajectory')
        pypl.plot(XOUT[i,0], XOUT[i,1],'or',linewidth='3')
        B_graph = TheBoat(XOUT[i,0],XOUT[i,1],XOUT[i,2]);
        pypl.plot(B_graph[0,:],B_graph[1,:],'k',LineWidth='1.5')
 
        #  Face a l'amer.        
        #  if or((abs(ya(i,1)-Angle_temp(1,i))<1e-2),(abs(yb(i,1)-Angle_temp(1,i))<1e-2))
 
        if (math.fabs(ya[i]-Angle_temp[i])<1e-1):
            pypl.plot([XOUT[i,0],XOUT[i,0]+5*np.cos(XOUT[i,2]+Angle_temp[i])], [XOUT[i,1],XOUT[i,1]+5*np.sin(XOUT[i,2]+Angle_temp[i])], 'b')#,color='C_amer1',lineStyle=':',lineWidth='2')
        elif (math.fabs(yb[i]-Angle_temp[i])<1e-1):
            pypl.plot([XOUT[i,0],XOUT[i,0]+5*np.cos(XOUT[i,2]+Angle_temp[i])], [XOUT[i,1],XOUT[i,1]+5*np.sin(XOUT[i,2]+Angle_temp[i])],'g')#,color='C_amer2',lineStyle=':',lineWidth='2')
        else:
            pypl.plot([XOUT[i,0],XOUT[i,0]+5*np.cos(XOUT[i,2]+w*t[i])], [XOUT[i,1],XOUT[i,1]+5*np.sin(XOUT[i,2]+w*t[i])],'r')#,color='C_rotating_Part', lineStyle=':',lineWidth='2')
          
 
        #pypl.plot([XOUT[i,0],A[0]], [XOUT[i,1],A[1]])#,linewidth='1',color='C_Boat2Amer')    # droites bateau - amer
 
        #pypl.plot([XOUT[i,0],XOUT[i,0]+np.cos(XOUT[i,2])], [XOUT[i,1],XOUT[i,1]+np.sin(XOUT[i,2])])#,'k',linewidth='2')
 
        #pypl.plot([XOUT[i,0],XOUT[i,0]+np.cos(XOUT[i,2]+ya[i])],[XOUT[i,1],XOUT[i,1]+np.sin(XOUT[i,2]+ya[i])])#,color='C_amer1', linewidth='2')
        #pypl.plot([XOUT[i,0],XOUT[i,0]+np.cos(XOUT[i,2]+yb[i])],[XOUT[i,1],XOUT[i,1]+np.sin(XOUT[i,2]+yb[i])])#,color='C_amer2', linewidth='2')
 
        #pypl.plot([XOUT[i,0],B[0]], [XOUT[i,1],B[1]])#,linewidth='1',color='C_Boat2Amer')
        #pypl.draw()
        pypl.show()
    
#print(temp2)    
    

[ 3.5472403   3.54659529  3.54595772 ...,  0.0320361   0.02228049
  0.0125041 ]
[ 4.85077753  4.85008846  4.84940476 ...,  1.34762596  1.33859317
  1.32954841]
ya [ 3.5472403   3.54659529  3.54595772 ...,  0.0320361   0.02228049
  0.0125041 ] 
 yb [ 4.85077753  4.85008846  4.84940476 ...,  1.34762596  1.33859317
  1.32954841]
angle temp: [ 0.         0.05       0.1       ...,  3.9708114  4.0208114  4.0708114] 3601
S1 114
S2 113
max(ya) 6.28073080729 
 min(ya) 0.00119696660244 
 max(yb) 6.28007827369 
 min(yb) 0.00149846839068 
 moyenne ya 3.4092360202 
 moyenne yb 3.50719795707


S1 avant remove [0.34500000000000003, 0.35000000000000003, 0.35499999999999998, 0.35999999999999999, 0.97499999999999998, 0.97999999999999998, 0.98499999999999999, 0.98999999999999999, 1.615, 1.6200000000000001, 1.625, 1.6300000000000001, 2.2650000000000001, 2.27, 2.2749999999999999, 2.2800000000000002, 2.9199999999999999, 2.9250000000000003, 2.9300000000000002, 2.9350000000000001, 3.585, 3.5899999999999999, 3

In [ ]:
###########
#### Passage en coordonnees polaires:
#### (x,y,theta)--> (alpha1, alpha2, rho1,rho2) 

alpha1=np.zeros((len(t),1))
alpha2=np.zeros((len(t),1))
rho1=np.zeros((len(t),1))
rho2=np.zeros((len(t),1))
x1=np.zeros((len(t),1))
x2=np.zeros((len(t),1))
y1=np.zeros((len(t),1))
y2=np.zeros((len(t),1))


for i in range (len(t)):
    alpha1[i]=math.atan2(XOUT[i,1]-A[1],XOUT[i,0]-A[0])#-math.pi
    rho1[i]  =math.sqrt((XOUT[i,1]-A[1])**2+(XOUT[i,0]-A[0])**2)
    alpha2[i]=math.atan2(XOUT[i,1]-B[1],XOUT[i,0]-B[0])#-math.pi
    rho2[i]  =math.sqrt((XOUT[i,1]-B[1])**2+(XOUT[i,0]-B[0])**2)

     
    
for i in range (len(t)):
    x1[i]=rho1[i]*math.cos(alpha1[i])
    y1[i]=rho1[i]*math.sin(alpha1[i])
    x2[i]=rho2[i]*math.cos(alpha2[i])+B[0]
    y2[i]=rho2[i]*math.sin(alpha2[i])+B[1]
   

"""
    A convertir en Python

    #     hold on
    #     plot(alpha2(1,i)*cos(rho2(1,i)), alpha2(1,i)*sin(rho2(1,i)))
    #     drawnow

    
    
    figure(2)
    #axis('square')
    axis([-2,12,-5,12])
    hold on
    plot([A(1,1), B(1,1)],[A(2,1), B(2,1)],'--','Color','k')
    plot(A(1,1), A(2,1),'o','Color',C_amer1)
    plot(B(1,1), B(2,1),'o','Color',C_amer2)

    plot(XOUT(:,1),XOUT(:,2),'Color',C_trajectory,'LineWidth',2);
    plot(x1(:,1),y1(:,1),'Color','r','LineWidth',1);
    plot(x2(:,1),y2(:,1),'Color','y','LineWidth',0.5);
    legend('','','','Trajectoire -coor. originales','trajectoire -- coor. poliares w.r.t A1','trajectoire -- coor. poliares w.r.t A2')

    hold off
    ###
    ### FIN FIGURE
    #########
    
"""

In [ ]:
#####                         #####
### DYNAMIQUE EN COORD POLAIRES ###
#
#
#
    
# Pol_A_0 = coordonnee initiale en representation polaire w.r.t. A
Pol_A_0 = [[math.sqrt((XOUT[0,1]-A[1])**2+(XOUT[0,0]-A[0])**2)],
            [math.atan2(XOUT[0,1]-A[1],XOUT[0,0]-A[0])],                 
            [XOUT[0,2]]]               #pourquoi pas alpha1=math.atan2(XOUT[0,1]-A[1],XOUT[0,0]-A[0])-math.pi??




"""
A convertir

# x1 = rho1, x2 = alpha1, x3 = theta        
[TOUT,POL1OUT] = ode45(@(t,x) [v(t)*cos(x(3)- x(2)); 
                                v(t)*sin(x(3)- x(2))/x(1);
                                      u(t)             ],TOUT,Pol_A_0,options);
                                      
"""                                      


#Pol_B_0 = coordonnee initiale en representation polaire w.r.t. A
Pol_B_0 = [[math.sqrt((XOUT[0,1]-B[1])**2+(XOUT[0,0]-B[0])**2)],
            [math.atan2(XOUT[0,1]-B[1],XOUT[0,0]-B[0])],    #idem
            [XOUT[0,2]]]             #pourquoi pas alpha2=math.atan2(XOUT[0,1]-B[1],XOUT[0,0]-B[0])-math.pi ??


"""
A convertir

# x1 = rho2, x2 = alpha2, x3 theta        
[TOUT,POL2OUT] = ode45(@(t,x) [v(t)*cos(x(3)- x(2)); 
                                v(t)*sin(x(3)- x(2))/x(1);
                                      u(t)              ],TOUT,Pol_B_0,options);                               

                            
"""                                      



"""
A convertir

    figure(3)
    title('trajectoire donnee par la dynamique en coord. polaires')
    hold on
%    axis([-2,30,-5,12])
    plot([A(1,1), B(1,1)],[A(2,1), B(2,1)],'--','Color','k')
    plot(A(1,1), A(2,1),'o','Color',C_amer1)
    plot(B(1,1), B(2,1),'o','Color',C_amer2)
    plot(XOUT(:,1),XOUT(:,2),'Color',C_trajectory,'LineWidth',3);

    plot(POL1OUT(:,1).*cos(POL1OUT(:,2)),POL1OUT(:,1).*sin(POL1OUT(:,2)),'r','LineWidth',1.5)

    plot(POL2OUT(:,1).*cos(POL2OUT(:,2))+B(1,1),POL2OUT(:,1).*sin(POL2OUT(:,2))+B(2,1),'g','LineWidth',0.5)

    hold off

    Measure1_polA = mod(POL1OUT(:,2)-POL1OUT(:,3)+pi,2*pi); % Xi_1 = alpha1 + pi -Theta

                                      %
                                      %
                                      % 
    % % DYNAMIQUE EN COORD POLAIRES % %
    %%%%%       -------------     %%%%%

                     
"""            

                 

In [ ]:
####           #####
# CALCUL DES SORTIES
#
#
# 



theta = XOUT[:,2]
mes_1=np.zeros((len(theta),1))
mes_3=np.zeros((len(theta),1))

mes_2 = rho1.copy()
for i in range (len(theta)):
    mes_1[i,0] = math.pi - theta[i]+alpha1[i]
    mes_3[i,0] = math.pi - theta[i]+alpha2[i]
   
    
   
#code a convertir

####                            ####
# Mesures d'angle sans modulo 2*pi #
#mes_1_V2 = mes_1.copy() 
#indexes = find(diff(mes_1)<-0.5)
#if len(indexes)>0:
#    for i in range len(indexes):
#        mes_1_V2[indexes(i)+1:end]=(2*math.pi)+mes_1_V2[indexes(i)+1:end]
         
    
    
#indexes = find(diff(mes_1_V2)>0.5)
#if len(indexes)>0:
#    for i in range len(indexes):
#        mes_1_V2[indexes(i)+1:end]=-(2*math.pi)+mes_1_V2[indexes(i)+1:end]
    
    
#mes_3_V2 = mes_3.copy()
#indexes = find(diff(mes_3)<-0.5)
#if len(indexes)>0:
#    for i in range len(indexes):
#        mes_3_V2[indexes(i)+1:end)]=(2*math.pi)+mes_3_V2[indexes(i)+1:end]
        
    
#indexes = find(diff(mes_3_V2)>0.5)
#if len(indexes)>0:
#    for i in range len(indexes):
#        mes_3_V2[indexes(i)+1:end]=-(2*math.pi)+mes_3_V2[indexes(i)+1:end]


#Verifier dans la conversion si transposée nécessaire
#Measures = np.array([[mes_1_V2],
#                     [mes_2],
#                     [mes_3_V2]])    # Format : [mes_1;mes_2;mes_3]


Noisy = 1
St_dev1 =0.1
St_dev2 =1 
St_dev3 =0.1


AA = np.array([[0.2], 
               [0.4], 
               [0.3]])
    # 0 : kills the noise
    # 1: keeps the noise
    
    
    
    
#if Noisy: 
#    Noise =np.random.randn(len(Measures))
#    for i in range (0,3):
#        name = ['St_dev',num2str(i)]
#        Noise[i,:]=Noise[i,:]*eval(name) 
      
      #Attention aux indices de i !!!!
#    for i in range (1,len(Noise)):
#        Noise[:,i] = (np.ones(3,1)-AA)*Noise[:,i-1]+AA*Noise[:,i]
        # Noise i <- nouveau signal
        # Noise i-1 <- ancien signal

      
#    Measures = Measures + Noise
#    figure(44)
     
      
#    for i in range (0,3):
#        subplot(3,1,i)         ##Verification de indice lors de conversion
#        plot(TOUT,Measures[i,:])

            
                                  #
                                  #
                                  #
######        --------         ####


In [33]:
################             ######
### Forme normale d'observabilite #
#
#
#
zed=np.zeros((len(t),3))
for i in range (len(t)):
    zed[i,0]= math.pi-XOUT[i,2]+alpha1[i,0]
    zed[i,1]= rho1[i]
    zed[i,2]= math.pi-XOUT[i,2]+alpha2[i,0]
    


### ???    phi_exact(:,1)=(sin(z(:,1)).*z(:,4)).^2+(sin(z(:,3)).*z(:,2)).^2-2*sin(z(:,1)).*sin(z(:,3)).*cos(abs(z(:,1)-z(:,3))).*z(:,2).*z(:,4)-(B(1)*z(:,2).*z(:,4)).^2;




### A convertir
###                ### 
# Dynamique Normale  # 
###                ###
#[TOUT,NORM_OUT] = ode45(@(t,z) [v(t)*sin(z(1))/z(2)-u(t);
#                                -v(t)*cos(z(1));
#                                v(t)*sin(z(3))/AK(z(1),z(2),z(3),B)-u(t)],TOUT,zed(1,:),options);


#figure(4)
#title('Simulation forme normale VS changement de variables');
# --- #
#subplot(3,1,1)
#plot(TOUT,mod(zed(:,1),2*pi),'k')
#hold on
#plot(TOUT,mod(NORM_OUT(:,1),2*pi),'r')
#hold off
# --- #
#subplot(3,1,2)
#plot(TOUT,zed(:,2),'k')
#hold on
#plot(TOUT,NORM_OUT(:,2),'r')
#hold off
# --- #
#subplot(3,1,3)
#plot(TOUT,mod(zed(:,3),2*pi),'k')
#hold on
#plot(TOUT,mod(NORM_OUT(:,3),2*pi),'r')
#hold off


#    figure(4)
#    plot(phi_exact)
#    title('\phi=\rho_1^2+\rho_2^2-2*\rho_1*\rho_2cos(\xi)-xb avec z exact' )
    ### for i in range (len(t)):
    ###     z[i,0]=alpha1[i,0]-XOUT[i,2]
    ###     z[i,1]=math.sin(XOUT[i,2]-alpha1[i,0])/rho1[i,0]
    ###     z[i,2]=alpha2[i,0]-XOUT[i,2]
    ###     z[i,3]=math.sin(XOUT[i,2]-alpha2[i,0])/rho2[i,0]



NameError: name 'alpha1' is not defined

In [ ]:
    ### ------------------------------------ ###
    ### changement de variable inverse.
    #
    #
    #
    #
    #
    
    z = NORM_OUT
    
    ### Assurons nous qu'il est possible de reconstituer (x,y,theta) a partir de
    ### (z1, z2, z3,z4)

    ### A un instant t donne, le bateau est sur une courbe de niveau (un arc de
    ### cercle circonstrit au triangle A-B-Bateau. Le centre C de ce cercle est
    ### le point de concours des mediatrices de ce triangle et peut donc etre
    ### calcule a la main. 

    ### En utilisant cela ainsi que le fait que rho1 et rho2 soient les rayons
    ### de cercles centres resp. en A et B et de rayon A-Bateau, resp.
    ### B-Bateau, on peut trouver une expression pour y et donc pour x et
    ### theta. On exploite notamment la connaissance de yA=theta-alpha1,
    ### yB=theta-alpha2 et rho1=sin(z1)/z2 (idem pour rho2)

    #diff(rho1,t)=v*cos(theta-alpha1)=v*cos(z1) (idem pour rho2)

    xi=np.zeros((len(t),1))
    C=np.zeros((len(t),2))
    R=np.zeros((len(t),1))
    r1=np.zeros((len(t),1))      #calcul de rho1 a partir de z1 et z2
    r2=np.zeros((len(t),1))      #calcul de rho1 a partir de z3 et z4
    y=np.zeros((len(t),1))
    x=np.zeros((len(t),1))
    a1=np.zeros((len(t),1))      #calcul de alpha1 et alpha2 a partir de x et y
    a2=np.zeros((len(t),1))
    tht=np.zeros((len(t),1))     #calcul de theta a partir de z1 et alpha1 (ou z3 et alpha2)

    
    for i in range (len(t)):
        xi[i,0]=abs(z[i,2]-z[i,0])
        C[i,0]=(B[1]-A[1]+math.tan(xi[i,0])*(A[0]+B[0]))/(2*math.tan(xi[i,0])) #pourquoi C(1) est il constant?
        C[i,1]=(B[0]-A[0]+math.tan(xi[i,0])*(A[1]+B[1]))/(2*math.tan(xi[i,0]))


# Attention verifier adaptation code        
    for i in range (len(C)):
        Ray[i,0] = math.sqrt(C[i,:]*np.transpose(C[i,:]))

    #  Ray(:,1)=sqrt((B(1,1)-A(1,1))^2+(B(2,1)-A(2,1))^2)./(2*sin(xi(:,1)));
    #r1(:,1)=sin(pi-z(:,1))./z(:,2);           
    for i in range (len(z)):
        r1 = z[i,1]
#    r2(:,1)=sin(pi-z(:,3))./z(:,4);        
    for i in range (len(r1)):
        r2[i,0] = AK(z[i,0],z[i,1],z[i,2],B)
    

    for i in range (len(t)):
        yNUM1[i] = (C[i,0]*(r2[i,0]**2-r1[i,0]**2)+B[0]*r1[i,0]**2)

        yDEN1[i] =(2*(B[0]*C[i,1]-C[i,0]*B[1]))

        yNUM2[i] = (B[0]*C[i,0]*(C[i,0]-B[0])-B[0]*Ray[i,0]**2+B[0]*C[i,1]**2-C[i,0]*B[1]**2)

        yDEN2[i] =(2*(B[0]*C[i,1]-C[i,0]*B[1]))

        
    y =yNUM1/yDEN1 + yNUM2/yDEN2

    
    for i in range (len(t)):
        x[i,0]=math.sqrt(r1[i,0]**2-y[i,0]**2)
        a1[i,0]=(math.atan2(A[1]-y[i,0],A[0]-x[i,0])-math.pi) % (2*math.pi)
        a2[i,0]=(math.atan2(B[1]-y[i,0],B[0]-x[i,0])-math.pi) % (2*math.pi)
        tht[i,0]=math.pi-z[i,0]+a1[i,0]

"""
    figure(5)
    title('Trajectoire reconstruite a partir de la forme normale')
    hold on
    plot(x(:,1),y(:,1),'r')
    plot(XOUT(:,1),XOUT(:,2),'Color',C_trajectory)
    hold off

"""


    ### Calcul erreur 
    for i in range (len(t)):
        erreur_rho[i,0]=r1[i,0]-rho1[i,0]
        erreur_alpha1[i,0]=alpha1[i,0]-a1[i,0]
        erreur_alpha2[i,0]=alpha2[i,0]-a2[i,0]

        erreur_x[i,0]=XOUT[i,0]-x[i,0]
        erreur_y[i,0]=XOUT[i,1]-y[i,0]
        erreur_theta[i,0]=XOUT[i,2]-tht[i,0]

"""
    figure(6)
    hold on
    title('erreur de reconstruction')
    plot(erreur_alpha2)
    plot(erreur_alpha1)
    plot(erreur_rho )
    plot(erreur_theta)
    #plot(erreur_x)
    #plot(erreur_y)
    hold off
  
"""
                                            
                                               #
                                               #
                                               #
                                               #
    ### changement de variable inverse.        #
    ### ------------------------------------ ###



In [ ]:
    #####                ---------------------               ####
    #  Asynchronous continuous-discrete High-Gain Kalman Filter #
    #                                                           #
    #####                ---------------------               ####
    
   
    TimeStamps = np.union1d(S1,S2)
    if TimeStamps[0]==S1[0]:
        FirstIndex = 1
    elseif TimeStamps[0]==S2[0]:
        FirstIndex = 2
    else
        error('Problem With first index')  # Verifier si existe en python
   

In [ ]:
    ####                          ###
    # Test d'alternance des indices #
    #
    #
    
    j=0
    for i in range (0,len(TimeStamps),2):
        First_indexes[j]  = i
        j+=1
    
    j=0    
    for i in range (1,len(TimeStamps),2):    
        Second_indexes[j] = i
        j+=1
    
    
 
    if FirstIndex == 1:
        if min(S1==TimeStamps(First_indexes)) & min(S2==TimeStamps(Second_indexes)):
            print('Alternance OK')
        else:
            print('Alternance NOT OK')
        
    else:
        if min(S1==TimeStamps(Second_indexes)) & min(S2==TimeStamps(First_indexes)):
            print('Alternance OK')
        else:
            print('Alternance NOT OK')
    
    
    
                                    #
                                    #   
    # Test d'alternance des indices #
    ####                          ###
    

In [ ]:
    ##### -------------------------
    ### Parametres de l'observateur    
    #
    #
    #
    #
    ### --------------
    # HIGH-GAIN PARAMETER
    #
    #
    HG = 3   # Grand-gain theta
    Delta=np.diag([1,1,1]) # Due to very basic normal form
        
    Q = 1e-2*np.diag([1,10,1])   # ???

#    Q=2e-2*np.eye(3,3)

    Qtheta=HG*np.linalg.inv(Delta)*Q*np.linalg.inv(Delta)
    
    R = np.diag([0.1,1,0.1])
    
#    R = 1e1*np.eye(3,3)
    

    
    
    # -----
    # Initial Guess
 #   xhat0 = np.array([[0.7],[6.3],[1.3]])  # Initial Guess In original coodinates
 #    ya = (math.atan2((A[1]-xhat0[1]),(A[0]-xhat0[0]))-xhat0[2]) % (2*math.pi)

    #mesure de l'angle entre l'orientation du bateau et la balise B
 #   yb = (math.atan2((B[1]-xhat0[1]),(B[0]-xhat0[0]))-xhat0[2]) % (2*math.pi)  
 #    RHO1 = math.sqrt(0.7**2+6.3**2)   
 #    xhat0 =np.array([[ya],[RHO1],[yb]])
      

    xhat0=np.transpose(zed[0])+np.array([[1],[-1],[-0.5]])   # Initial Guess in Normal coordinates
    
    # Riccati Initial State 
    As = (A_mat(0)+Db_mat(0,xhat0))
    P0=control.care(np.transpose(As),np.eye(3,3),Qtheta,R,np.zeros(3,3),np.eye(3,3)) # cf. cahier de Nico
    
    #S0=np.diag([1,1,1])       
    S0 = np.linalg.inv(P0)
    
    S0=SymReshape(S0)    ### A recuperer dans le mail d'Aida
    #
    #
    ###
                                      #
                                      #
    # Parametres de l'observateur     #
    ######### -------------------------    
    
    
    
    

In [ ]:
    sensor = FirstIndex
    Big_TOUT = []
    KALOUT2 = []
    for i in range(len(TimeStamps)):
        if i==0: 
            TSPAN = [0 ,TimeStamps[0]]
#            [TOUT2,KALOUT] = ode45(@(t,Z) Kalman_Pred_Step(t,Z,HG, Qtheta),TSPAN,[xhat0;S0],options); #convertir ode45
#            KALOUT2 = np.array([[KALOUT2],[KALOUT(1:end,:)]])
#            Big_TOUT = np.array([[Big_TOUT],[TOUT2(1:end,:)]]) 
           
        else:
            TSPAN = [TimeStamps[i-1], TimeStamps[i]]
#             [TOUT2,KALOUT] = ode45(@(t,Z) Kalman_Pred_Step(t,Z,HG, Qtheta),TSPAN,KALOUT2(end,:),options);  #convertir ode45
#            Big_TOUT = np.array([[Big_TOUT],[TOUT2(2:end,:)]])      
#            KALOUT2 = np.array([[KALOUT2],[KALOUT(2:end,:)]])       
        
        
        if ((i == 0) or (i == 1)):
            DT = TimeStamps[i]
        else:
            DT = TimeStamps[i]-TimeStamps[i-2]
            
            
        ###             ----------                   ###
        # In order to know which measure is considered #
           Where = max(find(TOUT<=TimeStamps[i]))      #        # verifier conversion find
        ###             ----------                   ###
        
        if FirstIndex==1:
           # -- MODEL RELATED -#
            C = np.array([[1,0,0], 
                          [0,1,0]])

#            Red = R(1:2,1:2)                    # verifier conversion
            
            # -- HIGH-GAIN CONSTRUCTION -#  
            
            IR_t=HG*np.linalg.inv(Red)
           
            # -- Measures -- #
#            Output_Signal = Measures(1:2,Where)      # verifier conversion
            
            # ----- #
            FirstIndex= (FirstIndex % 2)+1
        else:    
             # -- MODEL RELATED -#
            C = [0, 0, 1]

#            Red = R(3,3)                              # verifier conversion
            
            # -- HIGH-GAIN CONSTRUCTION -#  
            
            IR_t=HG*np.linalg.inv(Red)

            # -- Measures -- #
#            Output_Signal = Measures(3,Where)         # verifier conversion
            
            # ----- #
            FirstIndex= (FirstIndex % 2)+1
            

        ### Some vectors need to be transposes in order to meet the format
        ### of KALOUT2.
        
        Zminus = np.transpose(KALOUT2(end,1:3))                     # verifier conversion
        Sminus = SymReshape(np.transpose(KALOUT2(end,4:end)))       # verifier conversion
        
        Splus = Sminus + np.transpose(C)*IR_t*C*DT
         
        Zplus = Zminus-np.linalg.inv(Splus)*np.transpose(C)*IR_t*(C*Zminus-Output_Signal)*DT
        
        KALOUT2(end,1:3) = np.transpose(Zplus)                          # verifier conversion
        KALOUT2(end,4:end) = np.transpose(SymReshape(Splus))            # verifier conversion
        
  

'''
    figure(7) # Convergence dans les coordonnees normales
    
    #title('Simulation forme normale VS changement de variables')
    # --- #
    subplot(3,1,1)
    plot(TOUT,mod(NORM_OUT(:,1),2*pi),'k')
    hold on
    plot(TOUT,mod(zed(:,1),2*pi),'b')
    plot(Big_TOUT,mod(KALOUT2(:,1),2*pi),'r')
    hold off
    legend({'$z_1$','$\hat z_1$'},'Interpreter','latex')
    
    % --- %
    subplot(3,1,2)
    plot(TOUT,NORM_OUT(:,2),'k')
    hold on
    plot(TOUT,zed(:,2),'b')
    plot(Big_TOUT,KALOUT2(:,2),'r')
    hold off
    legend({'$z_2$','$\hat z_2$'},'Interpreter','latex')
    
    % --- %
    subplot(3,1,3)
    plot(TOUT,mod(NORM_OUT(:,3),2*pi),'k')
    hold on
    plot(TOUT,mod(zed(:,3),2*pi),'b')
    plot(Big_TOUT,mod(KALOUT2(:,3),2*pi),'r')
    hold off
    legend({'$z_3$','$\hat z_3$'},'Interpreter','latex')
    
'''      
        
        

In [ ]:
    ### ------------------------------------ ###
    ### changement de variable inverse (de l'etat estime)
    #
    #
    #
    #
    #
    
    z = KALOUT2(:,1:3)    # verifier la conversion
    
    ### Assurons nous qu'il est possible de reconstituer (x,y,theta) a partir de
    ### (z1, z2, z3,z4)

    ### A un instant t donne, le bateau est sur une courbe de niveau (un arc de
    ### cercle circonstrit au triangle A-B-Bateau. Le centre C de ce cercle est
    ### le point de concours des mediatrices de ce triangle et peut donc etre
    ### calcule a la main. 

    ### En utilisant cela ainsi que le fait que rho1 et rho2 soient les rayons
    ### de cercles centres resp. en A et B et de rayon A-Bateau, resp.
    ### B-Bateau, on peut trouver une expression pour y et donc pour x et
    ### theta. On exploite notamment la connaissance de yA=theta-alpha1,
    ### yB=theta-alpha2 et rho1=sin(z1)/z2 (idem pour rho2)

    #diff(rho1,t)=v*cos(theta-alpha1)=v*cos(z1) (idem pour rho2)    # verifier la conversion

    
    l = len(KALOUT2)
    
    xi=np.zeros((l,1))
    C=np.zeros((l,2))
    R=np.zeros((l,1))
    r1=np.zeros((l,1))      #calcul de rho1 a partir de z1 et z2
    r2=np.zeros((l,1))      #calcul de rho1 a partir de z3 et z4
    y=np.zeros((l,1))
    x=np.zeros((l,1))
    a1=np.zeros((l,1))      #calcul de alpha1 et alpha2 a partir de x et y
    a2=np.zeros((l,1))
    tht=np.zeros((l,1))     #calcul de theta a partir de z1 et alpha1 (ou z3 et alpha2)

    for i in range (len(t)):
        xi[i,0]=abs(z[i,2]-z[i,0])
        C[i,0]=(B[1]-A[1]+math.tan(xi[i,0])*(A[0]+B[0]))/(2*math.tan(xi[i,0])) #pourquoi C(1) est il constant?
        C[i,1]=(B[0]-A[0]+math.tan(xi[i,0])*(A[1]+B[1]))/(2*math.tan(xi[i,0]))


# Attention verifier adaptation code        
    for i in range (len(C)):
        Ray[i,0] = math.sqrt(C[i,:]*np.transpose(C[i,:]))

    #  Ray(:,1)=sqrt((B(1,1)-A(1,1))^2+(B(2,1)-A(2,1))^2)./(2*sin(xi(:,1)));
    #r1(:,1)=sin(pi-z(:,1))./z(:,2);           
    for i in range (len(z)):
        r1 = z[i,1]
#    r2(:,1)=sin(pi-z(:,3))./z(:,4);        
    for i in range (len(r1)):
        r2[i,0] = AK(z[i,0],z[i,1],z[i,2],B)


    for i in range (len(t)):
        yNUM1[i] = (C[i,0]*(r2[i,0]**2-r1[i,0]**2)+B[0]*r1[i,0]**2)

        yDEN1[i] =(2*(B[0]*C[i,1]-C[i,0]*B[1]))

        yNUM2[i] = (B[0]*C[i,0]*(C[i,0]-B[0])-B[0]*Ray[i,0]**2+B[0]*C[i,1]**2-C[i,0]*B[1]**2)

        yDEN2[i] =(2*(B[0]*C[i,1]-C[i,0]*B[1]))

        
    y =yNUM1/yDEN1 + yNUM2/yDEN2

    
    for i in range (len(t)):
        x[i,0]=math.sqrt(r1[i,0]**2-y[i,0]**2)
        a1[i,0]=(math.atan2(A[1]-y[i,0],A[0]-x[i,0])-math.pi) % (2*math.pi)
        a2[i,0]=(math.atan2(B[1]-y[i,0],B[0]-x[i,0])-math.pi) % (2*math.pi)
        tht[i,0]=math.pi-z[i,0]+a1[i,0]   

                       
              
                       
"""
    figure(8)
%    title('Trajectoire estimee par l''observateur')
    hold on
    plot(x(:,1),y(:,1),'r')
    plot(XOUT(:,1),XOUT(:,2),'Color',C_trajectory)
    hold off

    legend('Estimated state','Actual Trajectory')
"""                       
                                               #
                                               #                                               
                                               #
                                               #
    ### changement de variable inverse.        #
    ### ------------------------------------ ###    
    

In [8]:
A=np.array([[1],[-1],[-0.5],[3]])
B=A[0:3]
B

array([[ 1. ],
       [-1. ],
       [-0.5]])

In [40]:
zed[0]

array([ 0.,  0.,  0.])

In [29]:
np.diag([1,1,1])

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [6]:
range(0,10,2)

range(0, 10, 2)

In [13]:
a=True
b=True
if a and b:
    print('ok')
else:
    print('not ok')

ok


In [5]:
S1=np.array([[-1, 0, 1],[5,4,2]])
S2=np.array([[-2, 0, 2],[5,-2,7]])
np.union1d(S1,S2)

array([-2, -1,  0,  1,  2,  4,  5,  7])

In [12]:
S1=np.array([[-1, 0, 1],[5,4,2]])
[m,n]=S1.shape
print(m,n)

2 3


In [27]:
A=np.array([1,4,7])
B=np.array([1,4,7]) 
print(A==B)
min(A==B)

[ True  True  True]


True

In [ ]:
mes_1

In [ ]:
mes_1.shape

In [ ]:
theta.shape

In [ ]:
test=np.transpose(theta)
test.shape

In [ ]:
A.shape

In [ ]:
np.transpose(A).shape

In [ ]:
print(Angle_temp[106],ya[0], yb[0],A,B)
ii

In [ ]:
newS1=['%.3f' %elem for elem in S1]
%pprint
print(newS1)

In [ ]:
print('ii[52]',ii[52], '\n' ,
't[1651]',t[1651] ,'\n' ,
't[2393]',t[2393],'\n',
ya[1755],'\n',
     Angle_temp[1755])

In [ ]:
temp1

In [ ]:
L1=[0,1,2,3,8,9,10]
t0=L1
t1=L1.copy()
t2=L1[4]
print(L1,t0,t1,t2)
t1[4]+=2
t0[3]=0
print(L1,t0,t1,t2)
t2+=3
print(L1,t0,t1,t2)